<a href="https://colab.research.google.com/github/tomastastic/FinalProject/blob/main/SSL_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Source:
https://deeplearninguniversity.com/implementing-image-rotation-pretext-training-for-self-supervised-learning-using-keras/

# Importing libraries

In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras


# example of loading the cifar10 dataset
from matplotlib import pyplot
from keras.datasets import cifar10

# Loading Data

In [ ]:
# load dataset and perform train/test split
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [ ]:
#data=keras.datasets.cifar10
# Getting the Data
#(X_train, y_train), (X_test, y_test)=data.load_data()

In [ ]:
print("Length of X_train",len(X_train))
print("Length of y_train",len(y_train))
print("Length of X_test",len(X_test))
print("Length of y_test",len(y_test))


Length of X_train 50000
Length of y_train 50000
Length of X_test 10000
Length of y_test 10000


In [ ]:
# Normalizing the Dataset
X_train=X_train/255
X_test=X_test/255

In [ ]:
# Creating Validation and Test set from X,Y_test
X_val=X_test[:5000]
y_val=y_test[:5000]

X_test=X_test[5000:]
y_test=y_test[5000:]

In [ ]:
print("Length of X_val",len(X_val))
print("Length of y_val",len(y_val))
print("Length of X_test",len(X_test))
print("Length of y_test",len(y_test))

Length of X_val 5000
Length of y_val 5000
Length of X_test 5000
Length of y_test 5000


In [ ]:
# Creating Un-labeled dataset
X_unlabeled=X_train[10000:]

# Creating Labeled dataset
X_labeled=X_train[:10000]
y_labeled=y_train[:10000]

In [ ]:
print("Unlabeled:")
print("Length of X_unlabeled",len(X_unlabeled))
print("Labeled:")
print("Length of X_labeled",len(X_labeled))
print("Length of y_labeled",len(y_labeled))

Unlabeled:
Length of X_unlabeled 40000
Labeled:
Length of X_labeled 10000
Length of y_labeled 10000


#Preprocessing Data and creating a validation set

In [ ]:
# X_train_0 dataset will contain images rotated by 0 degrees(No rotation)
X_train_0=X_unlabeled.copy()

# X_train_90 dataset will contain images rotated by 90 degrees
X_train_90=np.rot90(X_unlabeled, axes=(1,2))

# X_train_180 dataset will contain images rotated by 180 degrees
X_train_180=np.rot90(X_unlabeled, 2, axes=(1,2))

# X_train_270 dataset will contain images rotated by 270 degrees
X_train_270=np.rot90(X_unlabeled, 3, axes=(1,2))

In [ ]:
print("Length of X_train_0",len(X_train_0))
print("Length of X_train_0,90,180,270 (Individually):",len(X_train_90))


Length of X_train_0 40000
Length of X_train_0,90,180,270 (Individually): 40000


In [ ]:
# Assigning pseudo-labels to rotated image datasets
y_train_0=np.full((40000), 0)
y_train_90=np.full((40000), 1)
y_train_180=np.full((40000), 2)
y_train_270=np.full((40000), 3)

In [ ]:
# Concatenating Datasets
X_train_unlabeled_full=np.concatenate((X_train_0, X_train_90, X_train_180, X_train_270), axis=0)
y_train_unlabeled_full=np.concatenate((y_train_0, y_train_90, y_train_180, y_train_270), axis=0)

In [ ]:
print("Length of X_train_unlabeled_full",len(X_train_unlabeled_full))
print("Length of y_train_unlabeled_full",len(y_train_unlabeled_full))


Length of X_train_unlabeled_full 160000
Length of y_train_unlabeled_full 160000


##These 3 must go, in favour of Andrew Ng standarized data methods:
 https://cs230.stanford.edu/blog/split/

In [ ]:
# The function will distribute the samples uniformly over dataset
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]


In [ ]:
# Randomly shuffling the concatenated dataset
X_train_unlabeled_full_shuffled, y_train_unlabeled_full_shuffled = (X_train_unlabeled_full, y_train_unlabeled_full)

In [ ]:
# Randomly shuffling the concatenated dataset
#X_train_unlabeled_full_shuffled, y_train_unlabeled_full_shuffled = unison_shuffled_copies(X_train_unlabeled_full, y_train_unlabeled_full)

# Creating a CNN and training it on rotations of image

In [ ]:
# Creating a Convolutional Neural Network
model = keras.models.Sequential([
        keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                            input_shape=[32, 32, 3]),
        keras.layers.MaxPooling2D(2),
        keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
        keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
        keras.layers.MaxPooling2D(2),
        keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
        keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
        keras.layers.MaxPooling2D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(4, activation="softmax")
])

In [ ]:
# Compiling the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

##DATA BITS START

In [ ]:
# Creating Validation and Test Dataset for Pretext Task
#X-validation,Y-Training
X_rot_val, X_rot_train = X_train_unlabeled_full_shuffled[:10000], X_train_unlabeled_full_shuffled[10000:]
#Y-validation,Y-Training
y_rot_val, y_rot_train = y_train_unlabeled_full_shuffled[:10000], y_train_unlabeled_full_shuffled[10000:]

In [ ]:
# Reshaping the Inputs
X_rot_val=X_rot_val.reshape(-1, 32, 32, 3)
X_rot_train=X_rot_train.reshape(-1, 32, 32, 3)

##DATA BITS END

In [ ]:
#WTF does this do?

tf.config.run_functions_eagerly(True)

NameError: ignored

In [ ]:
print("Length of X_rot_train",len(X_rot_train))
print("Length of y_rot_train",len(y_rot_train))
print("Length of X_rot_val",len(X_rot_val))
print("Length of y_rot_val",len(y_rot_val))

Length of X_rot_train 150000
Length of y_rot_train 150000
Length of X_rot_val 10000
Length of y_rot_val 10000


In [ ]:
# Training the model on Pretext Task
history = model.fit(X_rot_train, y_rot_train, epochs=5, validation_data=(X_rot_val, y_rot_val))

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/5
4688/4688 [==============================] - 3775s 805ms/step - loss: 1.2147 - accuracy: 0.4407 - val_loss: 1.2469 - val_accuracy: 0.3597
Epoch 2/5
4688/4688 [==============================] - 3784s 807ms/step - loss: 1.0996 - accuracy: 0.5110 - val_loss: 1.3223 - val_accuracy: 0.2951
Epoch 3/5
4688/4688 [==============================] - 3656s 780ms/step - loss: 1.0528 - accuracy: 0.5438 - val_loss: 1.1417 - val_accuracy: 0.4354
Epoch 4/5
Epoch 4/5
4688/4688 [==============================] - 4024s 858ms/step - loss: 0.9855 - accuracy: 0.5925 - val_loss: 1.3826 - val_accuracy: 0.3492
Epoch 5/5
Epoch 5/5
4688/4688 [==============================] - 3891s 830ms/step - loss: 0.9366 - accuracy: 0.6197 - val_loss: 1.0609 - val_accuracy: 0.5434


In [ ]:
# Checking the Model Architecture
model.summary()

In [ ]:
#SAVE MODEL
from google.colab import drive
drive.mount('/content/gdrive')
model_name = "Unsupervised"
path = f"/content/gdrive/My Drive/A Final Project/Dev/{model_name}"
model.save(path)

In [ ]:
# Removing the top layer and addding a new top layer
model.pop()
model.add(keras.layers.Dense(10, name='dense_3', activation='softmax'))

In [ ]:


# Checking the changes in Model Architecture
model.summary()

In [ ]:
# Check the 'Trainable' status of each layer
for layer in model.layers:
    print(layer.name, layer.trainable)

In [ ]:
# Freezing the Convolutional Layers while keeping Dense layers as Trainable
for layer in model.layers:
    if layer.name in ['dense_3', 'dense_1', 'dense', 'dropout', 'dropout_1']:
      layer.trainable=True
    else:
      layer.trainable=False

In [ ]:
# Checking if the changes in 'Trainable' status of each layer have taken place
for layer in model.layers:
    print(layer.name, layer.trainable)

conv2d False
max_pooling2d False
conv2d_1 False
conv2d_2 False
max_pooling2d_1 False
conv2d_3 False
conv2d_4 False
max_pooling2d_2 False
flatten False
dense True
dropout True
dense_1 True
dropout_1 True
dense_3 True
conv2d False
max_pooling2d False
conv2d_1 False
conv2d_2 False
max_pooling2d_1 False
conv2d_3 False
conv2d_4 False
max_pooling2d_2 False
flatten False
dense True
dropout True
dense_1 True
dropout_1 True
dense_3 True


#DATA BITS START

In [ ]:
# Reshaping the Inputs
X_labeled=X_labeled.reshape(-1, 32, 32, 3)
X_val=X_val.reshape(-1, 32, 32, 3)
X_test=X_test.reshape(-1, 32, 32, 3)

In [ ]:
# Compiling the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

# Training the model on Downstream Task
history=model.fit(X_labeled, y_labeled, validation_data=(X_val, y_val), epochs=10)

Epoch 1/10
313/313 [==============================] - 328s 1s/step - loss: 2.1469 - accuracy: 0.1927 - val_loss: 2.0403 - val_accuracy: 0.2796
Epoch 2/10
313/313 [==============================] - 321s 1s/step - loss: 2.0815 - accuracy: 0.2226 - val_loss: 1.9831 - val_accuracy: 0.3008
Epoch 3/10
313/313 [==============================] - 328s 1s/step - loss: 2.0228 - accuracy: 0.2373 - val_loss: 1.9099 - val_accuracy: 0.3266
Epoch 4/10
313/313 [==============================] - 328s 1s/step - loss: 1.9750 - accuracy: 0.2540 - val_loss: 1.8636 - val_accuracy: 0.3430
Epoch 5/10
313/313 [==============================] - 314s 1s/step - loss: 1.9356 - accuracy: 0.2676 - val_loss: 1.8113 - val_accuracy: 0.3596
Epoch 6/10
313/313 [==============================] - 321s 1s/step - loss: 1.9064 - accuracy: 0.2767 - val_loss: 1.7874 - val_accuracy: 0.3664
Epoch 7/10
313/313 [==============================] - 332s 1s/step - loss: 1.8655 - accuracy: 0.2982 - val_loss: 1.7509 - val_accuracy: 0.3734

In [ ]:
#SAVE Superv MODEL
#Source: https://machinelearningmastery.com/save-load-keras-deep-learning-models/#:~:text=The%20weights%20are%20saved%20directly,the%20symmetrical%20load_weights()%20function.
from google.colab import drive
drive.mount('/content/gdrive')
model_name = "supervised.h5"
path = f"/content/gdrive/My Drive/A Final Project/Dev/{model_name}"
model.save(path)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Evaluating the model on the Test set
model.evaluate(X_test, y_test)

157/157 [==============================] - 39s 246ms/step - loss: 1.7034 - accuracy: 0.3840


[1.703398585319519, 0.3840000033378601]